# جلسه ۲ – ارزیابی RAG با ragas

ارزیابی یک خط لوله RAG حداقلی با استفاده از معیارهای ragas: مرتبط بودن پاسخ، صحت، دقت زمینه.


# سناریو
این سناریو یک خط لوله حداقلی برای بازیابی و تولید (RAG) را به صورت محلی ارزیابی می‌کند. ما:
- یک مجموعه سند مصنوعی کوچک تعریف می‌کنیم.
- اسناد را جاسازی کرده و یک بازیاب شباهت ساده پیاده‌سازی می‌کنیم.
- پاسخ‌های مبتنی بر داده‌ها را با استفاده از یک مدل محلی (Foundry Local / سازگار با OpenAI) تولید می‌کنیم.
- معیارهای ragas (`پاسخ‌گویی مرتبط`, `صحت`, `دقت زمینه`) را محاسبه می‌کنیم.
- حالت FAST (متغیر محیطی `RAG_FAST=1`) را پشتیبانی می‌کنیم تا فقط مرتبط بودن پاسخ را برای تکرار سریع محاسبه کنیم.

از این دفترچه یادداشت استفاده کنید تا تأیید کنید که مدل محلی شما به همراه جاسازی‌ها، پاسخ‌های مبتنی بر داده‌های واقعی تولید می‌کند، پیش از اینکه به مجموعه‌های بزرگ‌تر گسترش دهید.


### توضیح: نصب وابستگی‌ها
کتابخانه‌های مورد نیاز را نصب می‌کند:
- `foundry-local-sdk` برای مدیریت مدل‌های محلی.
- رابط کاربری `openai`.
- `sentence-transformers` برای جاسازی‌های متراکم.
- `ragas` + `datasets` برای ارزیابی و محاسبه معیارها.
- آداپتور `langchain-openai` برای رابط LLM در ragas.

اجرای مجدد بدون مشکل؛ اگر محیط قبلاً آماده شده است، نیازی به اجرا نیست.


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### توضیح: وارد کردن‌های اصلی و معیارها  
کتابخانه‌های اصلی و معیارهای ragas بارگذاری می‌شوند. نکات کلیدی:  
- SentenceTransformer برای ایجاد embeddings.  
- `evaluate` + معیارهای انتخابی ragas.  
- `Dataset` برای ساخت مجموعه داده‌های ارزیابی.  
این وارد کردن‌ها هیچ فراخوانی از راه دوری را ایجاد نمی‌کنند (به جز احتمال بارگذاری کش مدل برای embeddings).  


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### توضیح: مجموعه داده کوچک و پاسخ‌های مرجع پرسش و پاسخ  
یک مجموعه کوچک در حافظه (`DOCS`)، مجموعه‌ای از سوالات کاربران و پاسخ‌های مرجع مورد انتظار را تعریف می‌کند. این موارد امکان محاسبه سریع و قطعی معیارها را بدون نیاز به دریافت داده‌های خارجی فراهم می‌کنند. در سناریوهای واقعی، شما از پرسش‌های تولیدی و پاسخ‌های تنظیم‌شده نمونه‌برداری می‌کنید.


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### توضیح: راه‌اندازی سرویس، تعبیه‌ها و وصله ایمنی
مدیر Foundry Local را راه‌اندازی می‌کند، یک وصله ایمنی برای انحراف طرحواره `promptTemplate` اعمال می‌کند، شناسه مدل را حل می‌کند، یک کلاینت سازگار با OpenAI ایجاد می‌کند و تعبیه‌های متراکم را برای مجموعه اسناد از پیش محاسبه می‌کند. این کار وضعیت قابل استفاده مجدد برای بازیابی + تولید را تنظیم می‌کند.


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### توضیح: تابع Retriever
یک بازیابی‌کننده ساده شباهت برداری را با استفاده از ضرب نقطه‌ای بر روی تعبیه‌های نرمال‌شده تعریف می‌کند. بالاترین k سندها را بازمی‌گرداند (به‌طور پیش‌فرض k=2). در محیط تولید، برای مقیاس‌پذیری و کاهش تأخیر، آن را با شاخص ANN (مانند FAISS، Chroma، Milvus) جایگزین کنید.


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### توضیح: تابع تولید
`generate` یک درخواست محدود ایجاد می‌کند (سیستم دستور می‌دهد که فقط از متن زمینه استفاده شود) و مدل محلی را فراخوانی می‌کند. دمای پایین (0.1) استخراج دقیق را به خلاقیت ترجیح می‌دهد. متن پاسخ کوتاه‌شده را بازمی‌گرداند.


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### توضیح: مقداردهی اولیه کلاینت پشتیبان  
اطمینان حاصل می‌کند که `client` وجود دارد حتی اگر سلول مقداردهی اولیه قبلی رد شده یا شکست خورده باشد—از خطای NameError در مراحل ارزیابی بعدی جلوگیری می‌کند.


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### توضیح: حلقه ارزیابی و معیارها
مجموعه داده ارزیابی را ایجاد می‌کند (ستون‌های مورد نیاز: question، answer، contexts، ground_truths، reference) و سپس معیارهای انتخاب‌شده ragas را تکرار می‌کند.

بهینه‌سازی:
- FAST_MODE محدود به مرتبط بودن پاسخ برای آزمایش‌های سریع است.
- حلقه هر معیار از محاسبه مجدد کامل جلوگیری می‌کند وقتی یک معیار شکست می‌خورد.

یک دیکشنری از معیار -> امتیاز (NaN در صورت شکست) خروجی می‌دهد.


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
